# BERT Training for Narrative Classification

This notebook demonstrates how to train a BERT model for narrative classification using the modular code structure.

In [1]:
import os
import sys
import pandas as pd
import logging
from datetime import datetime

# Import our custom modules
from dataset import prepare_data, CustomDataset
from model import predict, initialize_model
from trainer import train_bert
from modules.utils import debug_misclassifications, setup_logging


In [ ]:
setup_logging()
# Create logs directory if it doesn't exist
logs_dir = os.path.join(os.getcwd(), "code", "logs")
os.makedirs(logs_dir, exist_ok=True)

# Setup logging with specified directory
log_filename = os.path.join(logs_dir, f"preprocessing_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler(sys.stdout)
    ]
)

# Get logger
logger = logging.getLogger(__name__)

## Processing Summary Class

First, let's define our ProcessingSummary class to track and display results.

In [2]:
class ProcessingSummary:
    def __init__(self):
        self.start_time = datetime.now()
        self.steps_completed = []
        self.ml_results = {}
        self.document_stats = {}
        
    def add_step(self, step_name, details=None):
        step = {
            'name': step_name,
            'timestamp': datetime.now().strftime('%H:%M:%S'),
            'details': details
        }
        self.steps_completed.append(step)
    
    def add_ml_result(self, model_type, metrics):
        self.ml_results[model_type] = metrics
    def display_summary(self):
        duration = datetime.now() - self.start_time
        minutes = int(duration.total_seconds() // 60)
        seconds = int(duration.total_seconds() % 60)
        
        print("\n" + "="*80)
        print(f"{'PROCESSING SUMMARY':^80}")
        print("="*80)
        
        print("\nGENERAL INFORMATION")
        print("-"*80)
        print(f"Total Processing Time: {minutes}m {seconds}s")
        print(f"Steps Completed: {len(self.steps_completed)}")
        
        print("\nDOCUMENT STATISTICS")
        print("-"*80)
        for key, value in self.document_stats.items():
            if isinstance(value, float):
                print(f"{key.replace('_', ' ').title()}: {value:.2f}")
            else:
                print(f"{key.replace('_', ' ').title()}: {value}")
        
        print("\nPROCESSING TIMELINE")
        print("-"*80)
        for step in self.steps_completed:
            print(f"\n[{step['timestamp']}] {step['name']}")
            if step.get('details'):
                for key, value in step['details'].items():
                    print(f"  └─ {key}: {value}")
        
        if self.ml_results:
            print("\nML RESULTS")
            print("-"*80)
            for model, metrics in self.ml_results.items():
                print(f"\n{model}:")
                for metric, value in metrics.items():
                    if isinstance(value, float):
                        print(f"  └─ {metric}: {value:.4f}")
                    else:
                        print(f"  └─ {metric}: {value}")
        
        print("\n" + "="*80 + "\n")

# Initialize processing summary
summary = ProcessingSummary()

## Set Up Paths and Load Data

Define paths and load the preprocessed data files.

In [3]:
# Define paths
base_path = os.path.join(os.getcwd(), "..", "..")
output_dir = os.path.join(base_path, "outputs")
os.makedirs(output_dir, exist_ok=True)

# Load preprocessed data
try:
    df_normalized = pd.read_csv(os.path.join(base_path, "df_normalized.csv"))
    df_normalized_ua = pd.read_csv(os.path.join(base_path, "df_normalized_ua.csv"))
    df_normalized_cc = pd.read_csv(os.path.join(base_path, "df_normalized_cc.csv"))
    
    # Update summary
    summary.document_stats = {
        'total_documents': len(df_normalized),
        'ua_documents': len(df_normalized_ua),
        'cc_documents': len(df_normalized_cc)
    }
    
    summary.add_step('Data Loading', {
        'total_documents': len(df_normalized),
        'ua_documents': len(df_normalized_ua),
        'cc_documents': len(df_normalized_cc)
    })
    
except Exception as e:
    logger.error(f"Error loading data: {str(e)}")
    raise

## Training Options

Define functions to handle different training scenarios.

In [4]:
def get_ml_choice():
    print("\nSelect processing option:")
    print("1. Train BERT on all data")
    print("2. Train BERT on UA data only")
    print("3. Train BERT on CC data only")
    print("4. Run all BERT training variations")
    print("5. Skip training")

    while True:
        try:
            choice = int(input("\nEnter your choice (1-5): "))
            if 1 <= choice <= 5:
                return choice
            print("Please enter a number between 1 and 5.")
        except ValueError:
            print("Please enter a valid number.")

def run_selected_ml(choice, df_normalized, df_normalized_ua, df_normalized_cc, base_path, summary):
    if choice == 1:
        logger.info("Starting BERT training on full dataset...")
        training_results = train_bert(df_normalized, base_path)
        summary.add_ml_result('BERT (Full Dataset)', training_results)
        logger.info(f"BERT training on full data completed. Results: {training_results}")
    
    elif choice == 2:
        logger.info("Starting BERT training on UA dataset...")
        training_results_ua = train_bert(df_normalized_ua, base_path)
        summary.add_ml_result('BERT (UA Dataset)', training_results_ua)
        logger.info(f"BERT training on UA data completed. Results: {training_results_ua}")
    elif choice == 3:
        logger.info("Starting BERT training on CC dataset...")
        training_results_cc = train_bert(df_normalized_cc, base_path)
        summary.add_ml_result('BERT (CC Dataset)', training_results_cc)
        logger.info(f"BERT training on CC data completed. Results: {training_results_cc}")
    
    elif choice == 4:
        logger.info("Starting BERT training on all variations...")
        
        training_results = train_bert(df_normalized, base_path)
        summary.add_ml_result('BERT (Full Dataset)', training_results)
        logger.info(f"BERT training on full data completed. Results: {training_results}")
        
        training_results_ua = train_bert(df_normalized_ua, base_path)
        summary.add_ml_result('BERT (UA Dataset)', training_results_ua)
        logger.info(f"BERT training on UA data completed. Results: {training_results_ua}")
        
        training_results_cc = train_bert(df_normalized_cc, base_path)
        summary.add_ml_result('BERT (CC Dataset)', training_results_cc)
        logger.info(f"BERT training on CC data completed. Results: {training_results_cc}")

## Run Training

Select and execute the desired training option.

In [5]:
# Get user input for ML approach
choice = get_ml_choice()

# Run selected ML approach
if choice != 5:
    run_selected_ml(choice, df_normalized, df_normalized_ua, df_normalized_cc, base_path, summary)
else:
    logger.info("Training skipped as per user choice")
    summary.add_step('Training Skipped')

# Display summary
summary.display_summary()


Select processing option:
1. Train BERT on all data
2. Train BERT on UA data only
3. Train BERT on CC data only
4. Run all BERT training variations
5. Skip training
2025-01-20 21:20:40,927 - __main__ - INFO - Starting BERT training on all variations...


wandb: Currently logged in as: jonaskruse (backpropagandists). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


2025-01-20 21:20:42,462 - dataset - INFO - Number of unique labels in mapping: 75
2025-01-20 21:20:42,463 - dataset - INFO - Sample text: ['найтоп', 'септември', 'екстремен', 'горещин', 'гориво', 'хора', 'екосистем', 'умират', 'северен', 
2025-01-20 21:20:42,464 - dataset - INFO - Sample label: 0
2025-01-20 21:20:42,475 - dataset - INFO - Number of unique labels in mapping: 75
2025-01-20 21:20:42,476 - dataset - INFO - Sample text: ['russia', 'ukraine', 'war', 'मसक', 'यकरन', 'रत', 'भर', 'बरसए', 'डरन', 'मसइला', 'ठकन', 'तबह', 'वयकत'
2025-01-20 21:20:42,478 - dataset - INFO - Sample label: 72


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Confusion Matrix
1,3.579900,3.422873,0.239645,0.120862,0.082008,0.239645,"{'Class_0': [[214, 99], [11, 14]], 'Class_1': [[335, 0], [3, 0]], 'Class_2': [[336, 0], [2, 0]], 'Class_3': [[275, 50], [3, 10]], 'Class_4': [[331, 0], [7, 0]], 'Class_5': [[337, 0], [1, 0]], 'Class_6': [[334, 0], [4, 0]], 'Class_7': [[334, 0], [4, 0]], 'Class_8': [[327, 0], [11, 0]], 'Class_9': [[331, 0], [7, 0]], 'Class_10': [[330, 0], [8, 0]], 'Class_11': [[337, 0], [1, 0]], 'Class_12': [[336, 0], [2, 0]], 'Class_13': [[337, 0], [1, 0]], 'Class_15': [[336, 0], [2, 0]], 'Class_16': [[337, 0], [1, 0]], 'Class_17': [[337, 0], [1, 0]], 'Class_18': [[335, 0], [3, 0]], 'Class_19': [[337, 0], [1, 0]], 'Class_20': [[336, 0], [2, 0]], 'Class_21': [[333, 0], [5, 0]], 'Class_22': [[332, 0], [6, 0]], 'Class_24': [[337, 0], [1, 0]], 'Class_25': [[325, 0], [13, 0]], 'Class_26': [[330, 0], [8, 0]], 'Class_27': [[337, 0], [1, 0]], 'Class_28': [[335, 0], [3, 0]], 'Class_29': [[333, 0], [5, 0]], 'Class_30': [[335, 0], [3, 0]], 'Class_31': [[332, 0], [6, 0]], 'Class_32': [[336, 0], [2, 0]], 'Class_33': [[335, 0], [3, 0]], 'Class_34': [[322, 0], [16, 0]], 'Class_35': [[336, 0], [2, 0]], 'Class_36': [[331, 0], [7, 0]], 'Class_37': [[337, 0], [1, 0]], 'Class_38': [[334, 0], [4, 0]], 'Class_39': [[336, 0], [2, 0]], 'Class_40': [[337, 0], [1, 0]], 'Class_41': [[334, 0], [4, 0]], 'Class_42': [[337, 0], [1, 0]], 'Class_43': [[337, 0], [1, 0]], 'Class_45': [[337, 0], [1, 0]], 'Class_46': [[336, 0], [2, 0]], 'Class_47': [[337, 0], [1, 0]], 'Class_48': [[337, 0], [1, 0]], 'Class_50': [[337, 0], [1, 0]], 'Class_51': [[336, 0], [2, 0]], 'Class_52': [[335, 0], [3, 0]], 'Class_53': [[334, 0], [4, 0]], 'Class_54': [[335, 0], [3, 0]], 'Class_55': [[165, 108], [8, 57]], 'Class_56': [[337, 0], [1, 0]], 'Class_58': [[337, 0], [1, 0]], 'Class_59': [[337, 0], [1, 0]], 'Class_60': [[337, 0], [1, 0]], 'Class_61': [[334, 0], [4, 0]], 'Class_62': [[326, 0], [12, 0]], 'Class_63': [[329, 0], [9, 0]], 'Class_64': [[332, 0], [6, 0]], 'Class_67': [[337, 0], [1, 0]], 'Class_68': [[335, 0], [3, 0]], 'Class_69': [[335, 0], [3, 0]], 'Class_70': [[332, 0], [6, 0]], 'Class_71': [[337, 0], [1, 0]], 'Class_72': [[332, 0], [6, 0]], 'Class_73': [[335, 0], [3, 0]], 'Class_74': [[335, 0], [3, 0]]}"
2,3.269800,3.293115,0.233728,0.124106,0.086097,0.233728,"{'Class_0': [[313, 0], [25, 0]], 'Class_1': [[335, 0], [3, 0]], 'Class_2': [[336, 0], [2, 0]], 'Class_3': [[304, 21], [4, 9]], 'Class_4': [[331, 0], [7, 0]], 'Class_5': [[337, 0], [1, 0]], 'Class_6': [[334, 0], [4, 0]], 'Class_7': [[334, 0], [4, 0]], 'Class_8': [[327, 0], [11, 0]], 'Class_9': [[331, 0], [7, 0]], 'Class_10': [[330, 0], [8, 0]], 'Class_11': [[337, 0], [1, 0]], 'Class_12': [[336, 0], [2, 0]], 'Class_13': [[337, 0], [1, 0]], 'Class_15': [[336, 0], [2, 0]], 'Class_16': [[337, 0], [1, 0]], 'Class_17': [[337, 0], [1, 0]], 'Class_18': [[335, 0], [3, 0]], 'Class_19': [[337, 0], [1, 0]], 'Class_20': [[336, 0], [2, 0]], 'Class_21': [[333, 0], [5, 0]], 'Class_22': [[332, 0], [6, 0]], 'Class_24': [[337, 0], [1, 0]], 'Class_25': [[228, 97], [4, 9]], 'Class_26': [[330, 0], [8, 0]], 'Class_27': [[337, 0], [1, 0]], 'Class_28': [[335, 0], [3, 0]], 'Class_29': [[333, 0], [5, 0]], 'Class_30': [[335, 0], [3, 0]], 'Class_31': [[332, 0], [6, 0]], 'Class_32': [[336, 0], [2, 0]], 'Class_33': [[335, 0], [3, 0]], 'Class_34': [[290, 32], [12, 4]], 'Class_35': [[336, 0], [2, 0]], 'Class_36': [[331, 0], [7, 0]], 'Class_37': [[337, 0], [1, 0]], 'Class_38': [[334, 0], [4, 0]], 'Class_39': [[336, 0], [2, 0]], 'Class_40': [[337, 0], [1, 0]], 'Class_41': [[334, 0], [4, 0]], 'Class_42': [[337, 0], [1, 0]], 'Class_43': [[337, 0], [1, 0]], 'Class_45': [[337, 0], [1, 0]], 'Class_46': [[336, 0], [2, 0]], 'Class_47': [[337, 0], [1, 0]], 'Class_48': [[337, 0], [1, 0]], 'Class_50': [[337, 0], [1, 0]], 'Class_51': [[336, 0], [2, 0]], 'Class_52': [[335, 0], [3, 0]], 'Class_53': [[334, 0], [4, 0]], 'Class_54': [[335, 0], [3, 

...

c:\Users\krona\.conda\envs\llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\krona\.conda\envs\llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\krona\.conda\envs\llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\krona\.conda\envs\llama\lib\sit


Metrics for Class 0:
Confusion Matrix:
[[63 15]
 [ 6 19]]
Precision: 0.5588
Recall: 0.7600
F1 Score: 0.6441

Metrics for Class 1:
Confusion Matrix:
[[100   0]
 [  3   0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for Class 2:
Confusion Matrix:
[[101   0]
 [  2   0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for Class 3:
Confusion Matrix:
[[71 19]
 [ 3 10]]
Precision: 0.3448
Recall: 0.7692
F1 Score: 0.4762

Metrics for Class 4:
Confusion Matrix:
[[102   0]
 [  1   0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for Class 5:
Confusion Matrix:
[[101   0]
 [  2   0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for Class 6:
Confusion Matrix:
[[101   0]
 [  2   0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for Class 8:
Confusion Matrix:
[[101   0]
 [  2   0]]
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000

Metrics for Class 9:
Confusion Matrix:
[[102   0]
 [  1   0]]
Precision: 0.0000
Recall: 0.0000
F1 Score

c:\Users\krona\.conda\envs\llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\krona\.conda\envs\llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\krona\.conda\envs\llama\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\krona\.conda\envs\llama\lib\sit

eval/accuracy,▁█████
eval/f1,▁█████
eval/loss,█▅▃▂▁▁
eval/precision,▁█████
eval/recall,▁█████
eval/runtime,▆▄█▄▂▁
eval/samples_per_second,▃▅▁▅▇█
eval/steps_per_second,▃▅▁▅▇█
train/epoch,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇█████
train/grad_norm,▂▅▃▃▃▁▂▂▁▂▂▂▂▂▃▂▃▁▃▃▂▂▁▂▂█


2025-01-20 21:35:08,371 - __main__ - INFO - BERT training on CC data completed. Results: {'eval_loss': 2.226285219192505, 'eval_accuracy': 0.4368932038834951, 'eval_f1': 0.32338412182867354, 'eval_precision': 0.2607121053979007, 'eval_recall': 0.4368932038834951, 'eval_confusion_matrix': {'Class_0': [[63, 15], [6, 19]], 'Class_1': [[100, 0], [3, 0]], 'Class_2': [[101, 0], [2, 0]], 'Class_3': [[71, 19], [3, 10]], 'Class_4': [[102, 0], [1, 0]], 'Class_5': [[101, 0], [2, 0]], 'Class_6': [[101, 0], [2, 0]], 'Class_8': [[101, 0], [2, 0]], 'Class_9': [[102, 0], [1, 0]], 'Class_10': [[102, 0], [1, 0]], 'Class_11': [[100, 0], [3, 0]], 'Class_12': [[102, 0], [1, 0]], 'Class_13': [[101, 0], [2, 0]], 'Class_14': [[98, 0], [5, 0]], 'Class_15': [[97, 0], [6, 0]], 'Class_17': [[102, 0], [1, 0]], 'Class_18': [[102, 0], [1, 0]], 'Class_19': [[102, 0], [1, 0]], 'Class_21': [[102, 0], [1, 0]], 'Class_22': [[101, 0], [2, 0]], 'Class_23': [[102, 0], [1, 0]], 'Class_24': [[102, 0], [1, 0]], 'Class_26': [[1

## Model Analysis and Debugging

Analyze model performance and debug misclassifications.

In [6]:
# Get the path to the latest trained model
current_date = datetime.now().strftime("%Y%m%d")
model_path = os.path.join(base_path, f"models/bert_{current_date}")

# Load the model and tokenizer
from model import load_model_and_tokenizer
model, tokenizer = load_model_and_tokenizer(model_path)

# Load label mapping
import json
with open(os.path.join(model_path, "label_mapping.json"), 'r') as f:
    label_mapping = json.load(f)

# Analyze misclassifications
misclassifications = debug_misclassifications(
    dataset=df_normalized,
    model=model,
    tokenizer=tokenizer,
    label_mapping=label_mapping,
    dataset_type="Training"
)

# Save misclassification analysis
misclassifications_path = os.path.join(output_dir, f"analysis/misclassifications_{current_date}.csv")
os.makedirs(os.path.dirname(misclassifications_path), exist_ok=True)
misclassifications.to_csv(misclassifications_path, index=False)

print("\nMisclassified Examples:")
display(misclassifications)
print(f"\nAnalysis saved to: {misclassifications_path}")


Misclassified Examples:


,narrative,predicted_label,actual_label,dataset_type
0,"{'narrative': 'Criticism of climate movement',...",29,5,Training
1,{'narrative': 'Questioning the measurements an...,29,31,Training
2,"{'narrative': 'Criticism of climate movement',...",29,8,Training
3,"{'narrative': 'Speculating war outcomes', 'sub...",29,-1,Training
4,"{'narrative': 'Praise of Russia', 'subnarrativ...",0,-1,Training
...,...,...,...,...
1366,"{'narrative': 'Speculating war outcomes', 'sub...",29,-1,Training
1367,"{'narrative': 'Praise of Russia', 'subnarrativ...",0,-1,Training
1368,"{'narrative': 'Amplifying Climate Fears', 'sub...",29,0,Training
1369,"{'narrative': 'Praise of Russia', 'subnarrativ...",0,-1,Training



Analysis saved to: c:\Users\krona\OneDrive - TU Wien\TU Wien\1. Semester\NLP\nlp_Backpropagandists_2024\code\dl_methods\bert\..\..\outputs\analysis/misclassifications_20250120.csv


## Description of Deep Leaning baseline and used methods

As Deep Learning model we used BERT. We have trained the model using labeled data and measured the performance of the model using the following metrics:
* Accuracy
* Recall,
* Precision
* F1 Score

Before training the model, we have normalized and tokenized the data.
1. Normalization and tokenization: Cleaning and tokenizing the narratives, so that the model can be trained.
2. Label mapping: Creating a mapping between all unique class labels and integers to meet BERTs' requirements for training

For the models training and testing, we used a 80% training / 20% testing split.
By splitting the narratives, we followed different approaches to be able to evaluate the differences between them:
* Handling both, Ukraine War and Climate Change narratives in one dataframe, so that the model could learn from all the data
* Splitting narratives into two dataframes, containing only Ukraine War narratives or only Climate Change narratives each
* Using stratification to improve the distribution of classes between training and testing sets.

# Analysis

When the narratives were split by topic, the metrics for Climate Change improved, while the metrics for the Ukraine War worsened.
Having all metrics of 0.5135 during the approach with all dataframes at the same time, it is a moderate performance.

After that, we used only Climate Change narratives and all resulting metrics got much better: 0.7143.
Using only Ukraine War narratives, we received 0.4167 for all metrics, which is the worst result, although it is the topic with the highest amount of narratives that we had for training.

### Analyzing single classes

To be able to analyze differences in prediction of different classes, we analyzed the confusion matrices for each class individually.

For both topics, we received a similar distribution for the confusion matrix of the class *Other*. Again for both topics, the model predicted class "Other" correctly as positive. It also predicted other classes to be from class *Other*.

In [ ]:
import matplotlib.pyplot as plt

images = ['cc_other.png', 'cc_all.png', 'ua_other.png', 'ua_all.png']
labels = ['Climate Change Class Other', 'Climate Change all remaining classes', 'Ukraine War class Other', 'Ukraine War all remaining classes']
img_base_path = '../../info/screens/'
fig, ax = plt.subplots(2, 2, figsize=(10, 8))

for i, (img_name, label) in enumerate(zip(images, labels)):
    row = i // 2
    col = i % 2
    img_path = img_base_path + '/' + img_name
    img = plt.imread(img_path)
    ax[row, col].imshow(img)
    ax[row, col].axis('off')
    ax[row, col].set_title(label)
plt.show()

#### Climate Change

For Climate Change narratives, we received for class *Other* the following Confusion Matrix:
[[0, 4], [0, 10]]

Calculating the metrics for this specific class:
* Accuracy: 71,43 %
* Precision: 71,43 %
* Recall: 100%
* F1 Score: 83,33 %

For all other classes, we received the following equal Confusion Matrix:
[[13, 0], [1, 0]]
* Accuracy: 92,86 %
* Precision: 0 %
* Recall: 0%
* F1 Score: 0 %


#### Ukraine War

For Ukraine War narratives, we received for class *Other* the following Confusion Matrix:
[[0, 14], [0, 10]]

Calculating the metrics for this specific class:
* Accuracy: 41,7 %
* Precision: 41,7 %
* Recall: 100%
* F1 Score: 58,8%

For all other classes, we received the following equal Confusion Matrix:
[[23, 0], [1, 0]]
* Accuracy: 95,8%
* Precision: 0%
* Recall: 0%
* F1 Score: 0%


For both approaches, we can see that all classes except of *Other* have no counts for *True Positives*, but a very high count for *True Negatives*. On the hand, we have the class *Other*, where all samples of class *Other* are classified correctly as *Other*, but still we have almost as many or even more narratives that were classified as *Other*, although they had a different class.

From those unequally distributed results we see that BERTs' prediction performance is best for class *Other* and no other class.

An issue that could cause that, can be a data imbalance. The class *Other* occurs most frequently in the dataset. From that, the model could have learned to predict especially this class mostly accurately, while it fails for all other classes.

## Possible solutions
Although we have used stratification to receive a better distribution of all classes, there are still some solutions that we should consider in the next part.

Apperently, the class *Other* is overrepresented in the data set. Since we only had less than 200 narratives available, one possible solution to improve the predictions is to use more data.
In the context of this task we also have narratives in other languages available that we can use to handle the imbalance.Having more data.

The issue could also be that  the class labeling was not carried out cleanly, so that we have the broad class *Other*, where most of the narratives belong to. Specifying the classes more accurate could help to balance the dataset.





In [ ]:
import sys
import os
import ast
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

base_path = os.path.dirname(os.path.abspath(os.getcwd()))
input_file_full = os.path.join(base_path, "df_normalized.csv")
dataset = pd.read_csv(input_file_full)
min_examples_per_class = 1
# Get all narratives and count their frequencies
all_narratives = dataset['narrative_subnarrative_pairs'].apply(
    lambda x: eval(x) if isinstance(x, str) else x).tolist()

# Count frequencies of each narrative
narrative_counts = {}
for narratives in all_narratives:
    if narratives:
        narrative_str = str(narratives[0])  # Use first narrative
        narrative_counts[narrative_str] = narrative_counts.get(narrative_str, 0) + 1

# Filter narratives that have enough examples
valid_narratives = {
    narrative: count
    for narrative, count in narrative_counts.items()
    if count >= min_examples_per_class
}

# Create label mapping only for valid narratives
label_mapping = {
    narrative: idx
    for idx, narrative in enumerate(sorted(valid_narratives.keys()))
}

# Load the pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_mapping))

def safe_eval_debug(x, idx):
    try:
        # Safely evaluate the string
        parsed = ast.literal_eval(x)
        return parsed
    except Exception as e:
        # Print only entries that fail parsing
        print(f"Index {idx}: Error parsing - {x}")
        print(f"Exception: {e}")
        return None

# Iterate through the dataset and debug only failed entries
for idx, x in enumerate(dataset['narrative_subnarrative_pairs']):
    result = safe_eval_debug(x, idx)
# Debug
print(dataset['narrative_subnarrative_pairs'].head(10))

# Prepare the dataset for predictions
texts = dataset['narrative_subnarrative_pairs'].apply(
    lambda x: safe_eval_debug(x, idx)[0]['narrative'] if safe_eval_debug(x, idx) and len(safe_eval_debug(x, idx)) > 0 else None
).dropna().tolist()

print(f"Number of valid texts: {len(texts)}")
print(f"Sample texts: {texts[:5]}")

def validate_and_debug(x, idx):
    try:
        # Attempt to evaluate and extract the first narrative
        return eval(x)[0] if isinstance(x, str) else None
    except Exception as e:
        # Print the index and value causing the issue
        print(f"Error at index {idx}: {x}")
        print(f"Exception: {e}")
        return None

print(f"Number of valid texts: {len(texts)}")
print(f"Sample texts: {texts[:5]}")

# Apply the function to check each entry
texts = []
for idx, x in enumerate(dataset['narrative_subnarrative_pairs']):
    result = validate_and_debug(x, idx)
    if result is not None:
        texts.append(result)

print(f"Number of valid texts: {len(texts)}")
print(f"Sample texts: {texts[:5]}")

# Debig: Ensure all texts are strings
texts = [str(text) for text in texts if text is not None]

print(f"Number of valid texts: {len(texts)}")
print(f"Sample texts: {texts[:5]}")



encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)

# Create a dataset for inference
input_ids = torch.tensor(encodings['input_ids'])
attention_mask = torch.tensor(encodings['attention_mask'])

# Predict using the model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Convert predictions to narrative strings
predicted_narratives = [list(label_mapping.keys())[list(label_mapping.values()).index(pred)] for pred in predictions]

# Add the predicted narratives to the dataset
dataset['predicted_narratives'] = predicted_narratives

print(dataset.head())

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from transformer import train_bert, prepare_data
from transformer import predict
from tqdm import tqdm
import json

base_path = os.path.dirname(os.path.abspath(os.getcwd()))
label_directory = os.path.join(base_path, "models", "bert_20250113", "label_mapping.json")
input_file_full = os.path.join(base_path, "predicted_dataframe.csv")
dataset = pd.read_csv(input_file_full)

In [ ]:
texts, labels, label_mapping = prepare_data(dataset)
print(f"Sample text: {texts[:3]}")
print(f"Sample label: {labels[:3]}")


training_results = train_bert(dataset, base_path)
print(f"Training Results: {training_results}")

In [ ]:
tqdm.pandas()
model_path = os.path.join(base_path, "models/bert_20250113")

# Predict for the entire dataset
def classify_row(row):
    try:
        predicted_label, _ = predict(row['tokens_normalized'], model_path)
        return predicted_label
    except Exception as e:
        print(f"Error for row {row.name}: {e}")
        return None

# Apply predictions to the dataset
dataset['predicted_narrative'] = dataset.progress_apply(classify_row, axis=1)
dataset.to_csv(os.path.join(base_path, "predicted_dataframe.csv"), index=False)

# Debugging
print(dataset[['content', 'narrative_subnarrative_pairs', 'predicted_narrative']].head())

## Qualitative analysis

Here, we will pick one class label and analyze why some articles are getting predicted and especially why most of
the news articles that are actually assigned to that class are not predicted to be in this class.

We will again choose one class for qualitative analysis which has at least one *True Positive* and some *False Negatives* in English and Russian both, that we can compare with each other.

### English

In [ ]:
# Filter dataset for English texts
english_dataset = dataset[dataset['language'] == 'EN']
russian_dataset = dataset[dataset['language'] == 'RU']

# Extract unique classes from filtered dataset
all_classes = set([str(narr) for narr in english_dataset['temp_narrative']])

# Initialize a list for class summaries
class_summary_list = []

for target_class in all_classes:
    # Define target class as a dictionary
    target_dict = eval(target_class)  # Convert back to dictionary

    # Define True Positives, False Negatives, True Negatives, False Positives
    english_dataset['True Positives'] = english_dataset.apply(
        lambda row: row['predicted_narrative'] == target_dict and row['temp_narrative'] == target_dict, axis=1
    )
    english_dataset['False Negatives'] = english_dataset.apply(
        lambda row: row['predicted_narrative'] != target_dict and row['temp_narrative'] == target_dict, axis=1
    )
    english_dataset['True Negatives'] = english_dataset.apply(
        lambda row: row['temp_narrative'] != target_dict and row['predicted_narrative'] != target_dict, axis=1
    )
    english_dataset['False Positives'] = english_dataset.apply(
        lambda row: row['temp_narrative'] != target_dict and row['predicted_narrative'] == target_dict, axis=1
    )

    # Summarize counts for the current class
    class_summary_list.append({
        'target_class': target_class,
        'TP': english_dataset['True Positives'].sum(),
        'FN': english_dataset['False Negatives'].sum(),
        'FP': english_dataset['False Positives'].sum(),
        'TN': english_dataset['True Negatives'].sum()
    })

    # Generate confusion matrix for the current class
    y_true = english_dataset['temp_narrative'].apply(lambda temp: temp == target_dict).astype(int)
    y_pred = english_dataset['predicted_narrative'].apply(lambda pred: pred == target_dict).astype(int)

    cm = confusion_matrix(y_true, y_pred)

    # Display confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
    disp.plot(cmap="Blues")
    plt.title(f"Confusion Matrix for Target Class: {target_class}")
    plt.show()

# Create a DataFrame for class summaries
class_summary = pd.DataFrame(class_summary_list)

# Display class summary
print("Class Summary (English Texts):")
print(class_summary)


We will choose class *narrative: Hidden plots by secret schemes of powerful groups, subnarrative: Climate agenda has hidden motives* for the qualitative analysis of english.

We will print out the needed columns of the texts that were predicted with a different label, although they had the label mentioned above. Also, we will print news articles where the model predicted the label correctly to inspect those outputs.

In [ ]:
analyzing_class = {'narrative': 'Hidden plots by secret schemes of powerful groups',
                'subnarrative': 'Climate agenda has hidden motives'}

# Filtering true positives for analyzing class
true_positive_english = english_dataset[
    (english_dataset['predicted_narrative'] == analyzing_class) &
    (english_dataset['temp_narrative'] == analyzing_class)]

# Filtering false negatives for analyzing class
false_negatives_english = english_dataset[
    (english_dataset['predicted_narrative'] != analyzing_class) &
    (english_dataset['temp_narrative'] == analyzing_class)]

print("True positive indexes:")
print(true_positive_english.index.tolist())

print("\nFalse negative indexes:")
print(false_negatives_english.index.tolist())

In [ ]:
print(true_positive_english.loc[153, "content"])

Reading through this news article, we can see that there are two sentences in the beginning, where the author already starts to point to the direction, the new regulations being met "under the guise of fighting 'climate change'". He also says that most dishwasers that are sold today already align with the planned regulations and that the government should focus on other topics, the meaning of the direction the text is focused to gets clear. Those two cases have probably led to the correct assumption, that there are hidden plots by powerful groups (the government).
Having the text part "under the guise" likely led to the assumption that the climate agenda has hidden motives.

We can assume that the texts the model was trained with contained words like "hidden", "climate agenda" or "regime", because those words are all contained in the news article that was predicted correctly by the model. Those words were also used to hold information which likely led the label human classify the text under that label.

Further, the sentence, that "Critics have also poked major holes in the regime´s claim [...]" also could contribute to the fact that the model predicted especially this narrative-subnarrative pair, as the governments attempt to reduce energy consumption is getting criticized.



### False negative content:

In [ ]:
print(false_negatives_english.loc[104, "content"])

In [ ]:
print(false_negatives_english.loc[293, "content"])

In the first article, there are some sentences that indicate the topic *Climate Change* slightly. Especially the sentence saying "Caroline van der Plas founded [the movement]" and the following part about "nitrogen hoax", "climate change lies" and "[...] buy up most of the farmers" should lead the model to predict the actual label, as those words and sentences shall be likely been used in other news articles with that true label for training.
Nevertheless, the second part of the article starts getting religious with text parts like "Accept Jesus Christ as our saviour" or "Read the bible, fast and pray" or "Amen". Those words and especially this topic and tone of writing does not anything contribute to the true label. Thus, this drift off the climate change topic could lead the model to assume the text to belong toa different label, especially if there were some religious texts that were predicted with Other-Other or a different label, where religious topics would fit into the label.

The second article contains little Climate Change information throughout the whole text, which is understandable if one knows what kind of meetup was in Dubai and why influential people are joining it. Parts like "Globalist oligarchs met [...]" or "Unelected Bond villains who want to decide our fate." and "Mega Rich Elite [...]" tries to frame the politicians as a powerful group in a pejorative way.
Still, the model did not predict this news article as the true label. One reason for that could be, that the article is written in a sarcastic and pejorative way, which could lead the model to predict the label Other-Other, because the climate change content of the text is a bit hidden in political and social concepts that need to be understood.

## Russian

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# First, ensure we're working with a copy
russian_dataset = russian_dataset.copy()

# Extract unique classes from filtered dataset
all_classes = set([str(narr) for narr in russian_dataset['temp_narrative']])

# Initialize a list for class summaries
class_summary_list = []

# Create figure with larger size for better visibility
plt.figure(figsize=(12, 8))

for target_class in all_classes:
    # Define target class as a dictionary
    target_dict = eval(target_class)  # Convert back to dictionary
    
    # Define True Positives, False Negatives, True Negatives, False Positives using .loc
    russian_dataset.loc[:, 'True Positives'] = russian_dataset.apply(
        lambda row: row['predicted_narrative'] == target_dict and row['temp_narrative'] == target_dict, axis=1
    )
    russian_dataset.loc[:, 'False Negatives'] = russian_dataset.apply(
        lambda row: row['predicted_narrative'] != target_dict and row['temp_narrative'] == target_dict, axis=1
    )
    russian_dataset.loc[:, 'True Negatives'] = russian_dataset.apply(
        lambda row: row['temp_narrative'] != target_dict and row['predicted_narrative'] != target_dict, axis=1
    )
    russian_dataset.loc[:, 'False Positives'] = russian_dataset.apply(
        lambda row: row['temp_narrative'] != target_dict and row['predicted_narrative'] == target_dict, axis=1
    )

    # Calculate metrics
    TP = russian_dataset['True Positives'].sum()
    FN = russian_dataset['False Negatives'].sum()
    FP = russian_dataset['False Positives'].sum()
    TN = russian_dataset['True Negatives'].sum()
    
    # Calculate additional metrics
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Summarize counts for the current class
    class_summary_list.append({
        'target_class': target_class,
        'TP': TP,
        'FN': FN,
        'FP': FP,
        'TN': TN,
        'Accuracy': round(accuracy, 3),
        'Precision': round(precision, 3),
        'Recall': round(recall, 3),
        'F1': round(f1, 3)
    })

    # Generate confusion matrix for the current class
    y_true = russian_dataset['temp_narrative'].apply(lambda temp: temp == target_dict).astype(int)
    y_pred = russian_dataset['predicted_narrative'].apply(lambda pred: pred == target_dict).astype(int)

    cm = confusion_matrix(y_true, y_pred)

    # Create a new figure for each confusion matrix
    plt.figure(figsize=(8, 6))
    
    # Display confusion matrix with percentages
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
    disp.plot(cmap="Blues", values_format='.0f')
    plt.title(f"Confusion Matrix for Target Class: {target_class}")
    
    # Add text annotations with percentages
    total = np.sum(cm)
    for i in range(2):
        for j in range(2):
            percentage = cm[i, j] / total * 100
            plt.text(j, i, f'\n{percentage:.1f}%', 
                    ha='center', va='center')
    
    plt.tight_layout()
    plt.show()

# Create a DataFrame for class summaries
class_summary = pd.DataFrame(class_summary_list)

# Reorder columns for better readability
column_order = ['target_class', 'TP', 'FN', 'FP', 'TN', 'Accuracy', 'Precision', 'Recall', 'F1']
class_summary = class_summary[column_order]

# Display class summary with better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print("\nClass Summary (Russian Texts):")
print(class_summary.to_string(index=False))

# Calculate and display macro averages
macro_metrics = {
    'Accuracy': class_summary['Accuracy'].mean(),
    'Precision': class_summary['Precision'].mean(),
    'Recall': class_summary['Recall'].mean(),
    'F1': class_summary['F1'].mean()
}

print("\nMacro Averages:")
for metric, value in macro_metrics.items():
    print(f"{metric}: {value:.3f}")

In [ ]:
analyzing_class_russian = {'narrative': 'Discrediting Ukraine', 'subnarrative': 'Discrediting Ukrainian government and officials and policies'}

# Filtering true positives for analyzing class
true_positive_russian = russian_dataset[
    (russian_dataset['predicted_narrative'] == analyzing_class_russian) &
    (russian_dataset['temp_narrative'] == analyzing_class_russian)]

# Filtering false negatives for analyzing class
false_negatives_russian = russian_dataset[
    (russian_dataset['predicted_narrative'] != analyzing_class_russian) &
    (russian_dataset['temp_narrative'] == analyzing_class_russian)]

print("True positive indexes:")
print(true_positive_russian.index.tolist())

print("\nFalse negative indexes:")
print(false_negatives_russian.index.tolist())

### True positive content

In [ ]:
print(true_positive_russian.loc[422, "content"])